In [186]:
import os

### Download Data From Kaggle

In [187]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [188]:
!kaggle competitions download -c quora-insincere-questions-classification -f train.csv
!kaggle competitions download -c quora-insincere-questions-classification -f test.csv
!kaggle competitions download -c quora-insincere-questions-classification -f sample_submission.csv

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [189]:
train_fname = './train.csv.zip'
test_fname = './test.csv.zip'
sub_fname = './sample_submission.csv.zip'

In [190]:
import pandas as pd
import numpy as np

In [191]:
df = pd.read_csv(train_fname)
test_df = pd.read_csv(test_fname)
sub_df = pd.read_csv(sub_fname)

In [192]:
df

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [193]:
test_df

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?
...,...,...
375801,ffff7fa746bd6d6197a9,How many countries listed in gold import in in...
375802,ffffa1be31c43046ab6b,Is there an alternative to dresses on formal p...
375803,ffffae173b6ca6bfa563,Where I can find best friendship quotes in Tel...
375804,ffffb1f7f1a008620287,What are the causes of refraction of light?


In [194]:
sub_df

,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0


In [195]:
SAMPLE_SIZE = 100_000
sample_df = df.sample(SAMPLE_SIZE)

In [196]:
sample_df

,qid,question_text,target
1155823,e27b21553d6122806dc6,How would your past self give a pep talk to yo...,0
172475,21bb0bce414c40616beb,"What is the novel ""The Imperfectionists"" about?",0
1255197,f5fb3b24b9bd48bf16ff,My parents don't allow me to learn the guitar ...,0
1211987,ed8c0a288830a0c4c4ab,Is developing an app a good step towards becom...,0
68222,0d61e1767f26eb58d4f8,What causes pain in my stomach every night?,0
...,...,...,...
1245001,f3fb8f37133b37b053b3,How do I refer to someone in a sentence when I...,0
685311,86398a33290f654533c1,Is it true that Muslims don't care about loyal...,1
1148224,e0f8910f4d5fa6f210cf,What are the real wages in Hungary?,0
1269534,f8ce84f9badd77723844,Will you lose your trademark if you use it as ...,0


In [197]:
sample_df['target'].value_counts()

0    93818
1     6182
Name: target, dtype: int64

### Convert text to TF-IDF vectors

In [198]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [200]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [201]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [202]:
english_stopwords = stopwords.words('english')

In [203]:
", ".join(english_stopwords)

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [204]:
stemmer = SnowballStemmer(language='english')

In [205]:
def tokenize(text):
  return [stemmer.stem(token) for token in word_tokenize(text)]

In [206]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=english_stopwords, max_features=1000)

In [207]:
%%time
vectorizer.fit(sample_df['question_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'veri', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  warnings.warn(


CPU times: user 32.3 s, sys: 111 ms, total: 32.4 s
Wall time: 32.8 s


TfidfVectorizer(max_features=1000,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function tokenize at 0x7f6a69605bd0>)

In [208]:
vectorizer.get_feature_names_out()[:100]

array(['!', '$', '%', '&', "'", "''", "'m", "'s", '(', ')', ',', '-', '.',
       '1', '10', '100', '12', '12th', '15', '2', '20', '2017', '2018',
       '3', '4', '5', '6', '7', '8', ':', '?', '[', ']', '``', 'abl',
       'abus', 'accept', 'accomplish', 'accord', 'account', 'achiev',
       'act', 'action', 'activ', 'actor', 'actual', 'ad', 'add', 'admiss',
       'adult', 'advanc', 'advantag', 'advic', 'affect', 'africa',
       'african', 'age', 'ago', 'air', 'allow', 'almost', 'alon',
       'alreadi', 'also', 'alway', 'amazon', 'america', 'american',
       'among', 'amount', 'analysi', 'ancient', 'android', 'ani', 'anim',
       'anoth', 'answer', 'anxieti', 'anyon', 'anyth', 'apart', 'app',
       'appear', 'appl', 'appli', 'applic', 'approach', 'arab', 'area',
       'armi', 'around', 'art', 'asian', 'ask', 'atheist', 'attack',
       'attract', 'australia', 'avail', 'averag'], dtype=object)

### Transform to Vectors

In [209]:
%%time
inputs = vectorizer.transform(sample_df['question_text'])

CPU times: user 31 s, sys: 95.7 ms, total: 31.1 s
Wall time: 31.3 s


In [210]:
inputs.shape

(100000, 1000)

In [211]:
%%time
test_inputs = vectorizer.transform(test_df['question_text'])

CPU times: user 2min 1s, sys: 403 ms, total: 2min 1s
Wall time: 2min 4s


In [212]:
test_inputs.shape

(375806, 1000)

### Training and Validation Sets

In [213]:
from sklearn.model_selection import train_test_split

In [214]:
targets = sample_df['target']

In [215]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, targets, test_size=0.3)

In [216]:
print(f"Train Input Shape: {train_inputs.shape}")
print(f"Validation Input Shape: {val_inputs.shape}")
print(f"Train Target Shape: {train_targets.shape}")
print(f"Validation Target Shape: {val_targets.shape}")

Train Input Shape: (70000, 1000)
Validation Input Shape: (30000, 1000)
Train Target Shape: (70000,)
Validation Target Shape: (30000,)


### Convert to Pytorch Tensors

In [217]:
import torch

In [218]:
train_input_tensors = torch.tensor(train_inputs.toarray()).float()
val_input_tensors = torch.tensor(val_inputs.toarray()).float()
test_input_tensors = torch.tensor(test_inputs.toarray()).float()

In [219]:
train_target_tensors = torch.tensor(train_targets.values)
val_target_tensors = torch.tensor(val_targets.values)

### Create Pytorch Data Loaders

In [220]:
from torch.utils.data import TensorDataset, DataLoader

In [221]:
train_data = TensorDataset(train_input_tensors, train_target_tensors)
val_data = TensorDataset(val_input_tensors, val_target_tensors)
test_data = TensorDataset(test_input_tensors)

In [222]:
BATCH_SIZE = 128

In [223]:
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_data, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

### Train the Deep Learning Model

In [224]:
import torch.nn as nn
import torch.nn.functional as F

In [225]:
from sklearn.metrics import accuracy_score, f1_score

In [226]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer1 = nn.Linear(1000, 512)
    self.layer2 = nn.Linear(512, 256)
    self.layer3 = nn.Linear(256, 128)
    self.layer4 = nn.Linear(128, 1)

  def forward(self, inputs):
    out = self.layer1(inputs)
    out = F.relu(out)
    out = self.layer2(out)
    out = F.relu(out)
    out = self.layer3(out)
    out = F.relu(out)
    out = self.layer4(out)
    
    return out

In [227]:
model = NeuralNetwork()

In [229]:
def evaluate(model, dl):

  losses, accuracies, f1s = [], [], []

  for batch in dl:
    ## Get inputs and targets
    inputs, targets = batch

    ## Pass Inputs through models
    outputs = model(inputs)

    ## Convert to Probabilities
    probs = torch.sigmoid(outputs[:,0])

    ## Compute Loss
    loss = F.binary_cross_entropy(probs, targets.float(), weight=torch.tensor(20.))
    losses.append(loss.item())

    ## Compute Preds
    preds = (probs > 0.5).int()

    ## Compute Accuracy and F1 Score
    accuracy = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds)

    accuracies.append(accuracy)
    f1s.append(f1)

  return torch.mean(torch.tensor(losses)).item(), torch.mean(torch.tensor(accuracies)).item(), torch.mean(torch.tensor(f1s)).item()

In [230]:
def fit(epochs, lr, model, train_dl, val_dl):
  track_list = []
  optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=1e-5)

  for epoch in range(epochs):
    for batch in train_dl:
      inputs, targets = batch

      outputs = model(inputs)

      probs = torch.sigmoid(outputs[:,0])

      loss = F.binary_cross_entropy(probs, targets.float(), weight=torch.tensor(20.))

      ## Optimization
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    
    ## Evaluation
    loss, acc, f1 = evaluate(model, val_dl)
    print(f'Epoch: {epoch+1}, Loss: {loss}, Accuracy: {acc}, F1 Score: {f1}')

    track_list.append([loss, acc, f1])

  return track_list


In [231]:
fit(10, 0.001, model, train_dl, val_dl)

Epoch: 1, Loss: 3.1593868732452393, Accuracy: 0.9422761524822696, F1 Score: 0.3776319138390148
Epoch: 2, Loss: 3.342284679412842, Accuracy: 0.9433178191489362, F1 Score: 0.27615886345976587
Epoch: 3, Loss: 3.3127522468566895, Accuracy: 0.9444703014184398, F1 Score: 0.40431246505930823
Epoch: 4, Loss: 3.7547669410705566, Accuracy: 0.9395500886524824, F1 Score: 0.4020665397268761
Epoch: 5, Loss: 5.149354934692383, Accuracy: 0.9390514184397164, F1 Score: 0.37325206425043106
Epoch: 6, Loss: 6.46133279800415, Accuracy: 0.9379321808510638, F1 Score: 0.37869906936066583
Epoch: 7, Loss: 7.51602029800415, Accuracy: 0.9372007978723405, F1 Score: 0.4026203255714421
Epoch: 8, Loss: 8.84241771697998, Accuracy: 0.9377216312056738, F1 Score: 0.3753936612831284
Epoch: 9, Loss: 9.140625, Accuracy: 0.9358599290780142, F1 Score: 0.37300303559090864
Epoch: 10, Loss: 11.126466751098633, Accuracy: 0.9383311170212766, F1 Score: 0.3904937435998631


[[3.1593868732452393, 0.9422761524822696, 0.3776319138390148],
 [3.342284679412842, 0.9433178191489362, 0.27615886345976587],
 [3.3127522468566895, 0.9444703014184398, 0.40431246505930823],
 [3.7547669410705566, 0.9395500886524824, 0.4020665397268761],
 [5.149354934692383, 0.9390514184397164, 0.37325206425043106],
 [6.46133279800415, 0.9379321808510638, 0.37869906936066583],
 [7.51602029800415, 0.9372007978723405, 0.4026203255714421],
 [8.84241771697998, 0.9377216312056738, 0.3753936612831284],
 [9.140625, 0.9358599290780142, 0.37300303559090864],
 [11.126466751098633, 0.9383311170212766, 0.3904937435998631]]

## Predicting 

### On Dataframe

In [232]:
def predict_df(df):

  inputs = vectorizer.transform(df['question_text'])
  input_tensors = torch.tensor(inputs.toarray()).float()
  outputs = model(input_tensors)
  probs = torch.sigmoid(outputs)[:,0]
  preds = (probs > 0.5).int()
  
  return preds

In [233]:
temp_df = df.sample(10)

In [234]:
temp_df

,qid,question_text,target
1053992,ce874ef2c28e150e2bd6,India's foreign relations are in tatters now. ...,0
676009,84632e574d48c532db21,What is the difference between the vaccum and ...,0
470581,5c24cd01a6d0cd61c8f8,If every single person on Earth suddenly becam...,0
180019,232fd25df8e4ce1c3e8e,What influenced Brad Womack to become an entre...,0
879174,ac3f8e61642b41085fe7,What is it like to work in market research?,0
1034348,caaf6f7190480b3c9fd1,The area of the square is 144 sq.m. what is th...,0
1224848,f00ce9f3f8826cf730f5,About how long dose a black bear live?,0
859405,a860d570f24b43070c6c,How do I deal with a dad that thinks he is a c...,0
597331,750062e8d5e4eb5384ec,What is the pH of 0.0004 m naoh?,0
80546,0fc71de0032e5444f56f,What are the best James Bond related jokes?,0


In [235]:
temp_df['question_text'].values

array(["India's foreign relations are in tatters now. US cancelled the 2+2 dialogue. Nikki Haley came to threaten India. Isn't this a blot on India's sovereignty?",
       'What is the difference between the vaccum and the space?',
       'If every single person on Earth suddenly became blind, would the human race go extinct?',
       'What influenced Brad Womack to become an entrepreneur?',
       'What is it like to work in market research?',
       'The area of the square is 144 sq.m. what is the length of its side?',
       'About how long dose a black bear live?',
       'How do I deal with a dad that thinks he is a computer genius, but is actually a computer illiterate?',
       'What is the pH of 0.0004 m naoh?',
       'What are the best James Bond related jokes?'], dtype=object)

In [236]:
temp_df['target'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [237]:
predict_df(temp_df)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

### On Text

In [238]:
def predict_text(text):

  df = pd.DataFrame({'question_text': [text]})

  inputs = vectorizer.transform(df['question_text'])
  input_tensors = torch.tensor(inputs.toarray()).float()
  outputs = model(input_tensors)
  probs = torch.sigmoid(outputs)[:,0]
  preds = (probs > 0.5).int()
  
  answer_in_int = preds.detach().numpy()[0]

  if answer_in_int == 1:
    answer = 'Insincere'
  else:
    answer = 'Sincere'

  return answer

In [239]:
predict_text("Which is actually worth achieving - materialistic achievements or spiritual achievements?")

'Sincere'

In [240]:
predict_text("Why is North Korea so intent on obtaining nuclear weapons? What is their goal?")

'Sincere'

In [241]:
predict_text("Why are some questions on here so stupid? Are they trolling, kids, or just stupid?")

'Insincere'

### Prediction on Test Data

In [242]:
def predict(model, dl):
    all_preds = []
    for batch in dl:
        inputs, = batch
        out = model(inputs)
        probs = torch.sigmoid(out)[:,0]
        preds = (probs > 0.5).int()
        all_preds += list(preds.numpy())
    return all_preds

In [243]:
test_preds = predict(model, test_dl)

In [244]:
sub_df['prediction'] = test_preds

In [245]:
sub_df

,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0


In [246]:
sub_df['prediction'].value_counts()

0    361072
1     14734
Name: prediction, dtype: int64

In [247]:
sub_df.to_csv('submission.csv', index=None)

## The End